In [ ]:
import tkinter
from tkinter import*

import nltk
from nltk.stem import SnowBallStemmer

import tensorflow
import tflearn

import random
import numpy as np
import pickle
import json

stemmer = SnowballStemmer("Spanish")

with open("intents.json") as file:
    data= json.load(file)

def bag_of_words(s,word):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tonkensize(s)
    
    s_words = [stemmer.sten(word.lower()) for word in s_words]
    
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1 
    
    return np.array(bag)


base: Tk()
base.tittle("Chatbot de GALAY")
base.geometry("400x500")
base.resizable(width= FALSE, height= FALSE)

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial")
ChatLog.config(foreground="black", font=("Verdana", 12))
ChatLog.insert(END, "Saludos, consulta lo que necesitas")
ChatLog.place(x=6, y=6, height=386, width=370)

scrollbar = Scrollbar(base, command= ChatLog.yview, cursor="arrow")
ChatLog["yscrollcomand"]= scrollbar.set
scrollbar.place(x=376, y=6, height=386)

ChatLog.config(state=DISABLED)

def chatbot_response(msg):
    try:
        with open("data.pickle", "rb") as f:
            words, labels, training, output = pickle.load(f)
            
            model.load("model.tflearn")
            
    except:
        
        words= []
        labels=[]
        docs_x=[]
        docs_y=[]

        
        for intents in data["intets"]:
            for patterns in intents["pattern"]:
                wrds = nltk.word_tokenize(patterns)
                words.extend(wrds)
                docs_x.append(wrds)
                docs_y.append(intents["tag"])
                
                if intents["tag"] not in labels:
                    labels.append(intents["tag"])
        
        words=[stemmer.stem(w.lower()) for w in words if w != "?"]
        words = sorted(list(set(words)))
        labels = sorted(labels)
        
        training=[]
        output=[]
        
        out_empty= [0 for _ in range (len(labels))]
        
        for x, doc in enumerate(docs_x):
            bag= []
            wrds= [stemmer.stem(w.lower()) for w in doc]
            
            for w in words: 
                if w in wrds:
                    bag.append(1)
                else:
                    bag.append(0)
                    
                output_row = out_empty[:]
                output_row[labels.index(docs_y[x])] = 1
                
                training.append(bag)
                output.append(output_row)
        
        training = np.array(training)
        output = np.array(output)
        
        with open ("data.pickle", "wb") as f:
            pickle.dump((words, labels, trainint, output), f)
            
        tensorflow.compat.v1.reset_default_graph()
        
        net = tflearn.input_data(shape=[None, len(training[0])])
        
        net = tflearn.fully_connected(net, 8)
        
        net = tflearn.fully_connected(net, 8)
        
        net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
        net = tflearn.regression(net)
        
        
        model = tflearn.DNN(net)
        
        try:
            model.load("model.tflearn")
        except:
            model.fit(training, output, n_epoch=90, batch_size=8, show_metric=True)
            model.save("model.tflearn")
        
        results = model.predict([bag_of_words(msg, words)])
        
        results_index = np.argmax(results)
        
        tag = labels[results_index]
        
        for tg in data["intents"]:
            if tg["tag"] == tag:
                responses = tg["responses"]
                
        return(random.choice(responses))

def send():
    msg=EntryBox.get("1.0", "end-1c").strip()
    EntryBox.delete("0.0", END)
    
    res=chatbot_response(msg)
    
    ChatLog.config(state=NORMAL)
    ChatLog.insert(END, "Tu : "+msg+"n\n")
    ChatLog.config(foreground="green", font=("verdana",11))
    ChatLog.insert(END, "ChatBot :"+res+"n\n")
    ChatLog.yview(END)
    
SendButton = Button(base, font=("verdana", 12, "bold"), text="Send", width=9,
                   height=5, bd=0, bg="red", activebackground="gold", 
                   fg= "#ffffff". command=send)
SendButton.place(x=282, y=401, height=90)

EntryBox= Text(base, bd=0, bg="White", width="29", height)
        